# Converting the CMIP6 Data Set to the OpenVisus Visualization Format

This Jupyter notebook provides an example of how one might take a portion of the CMIP6 Data Set and convert it to the OpenVisus Visualization Format.

## Assumptions

1. You are running this notebook via [OSG's OSPool Notebooks service](https://notebook.ospool.osg-htc.org) as a user of ap40.uw.osg-htc.org.

2. You have run through the steps in [the setup notebook](cmip6_setup.ipynb).

3. You have selected the `openvisuspy` kernel for this notebook via the Jupyter interface.

4. You have a token stored in the file `osdf.token` that authorizes the bearer to write into OSDF.

## Define the scenario to process and where to place the output

Set the following variables to the portion of the CMIP6 data set that you'd like to process.

In [ ]:
model = "ACCESS-CM2"
interval = "day"
scenario = "historical"
variant = "r1i1p1f1"
variant_2 = "gn"
variable = "hurs"
version = ""

Set the following variable to the path in OSDF where you'd like to place the converted data.

In [ ]:
output_path = "/nsdf/testing/20241114/v7"

Set the following variable to the location of the `openvisuspy` container image that you created in [the setup notebook](cmip6_setup.ipynb).

In [ ]:
container_image = "osdf:///ospool/ap40/data/brian.aydemir/openvisuspy-20241113-215324.sif"

We can now define the full URLs for the OpenVisus output. We use `pelican://` URLs for compatibility with the OpenVisus library.

In [ ]:
output_loc = f"pelican://osg-htc.org{output_path}"

idx_filename = "nex-gddp-cmip6.idx"
dataset_loc = f"{output_loc}/{idx_filename}"
dataset_path = f"{output_path}/{idx_filename}"

## Convert the data set's objects

### Determine the list of objects to process

In [ ]:
## Identify the objects in AWS that might need processing.

version_suffix = f"_{version}" if version else ""
aws_objects = !grep -E '{model}/{scenario}/{variant}/{variable}/{variable}_{interval}_{model}_{scenario}_{variant}_{variant_2}_([0-9]+){version_suffix}.nc' ../nex-gddp-cmip6-listing/nex-gddp-cmip6.txt
aws_objects = [x.split()[3] for x in aws_objects]


## Filter objects that have already been processed.

import os.path
import pelicanfs.core

osdf_fs = pelicanfs.core.PelicanFileSystem("pelican://osg-htc.org", direct_reads=True)

for path in aws_objects[:]:
    filename = os.path.basename(path)
    print(f"Checking: {filename}")
    if osdf_fs.exists(f"{output_path}/{filename}.idx"):
        aws_objects_list.remove(entry)

In [ ]:
## Further filter the list of files, e.g., for development and testing.

aws_objects = aws_objects[:1]

### Submit the conversion jobs

In [ ]:
import pathlib
import platform
import htcondor


## HACK: Ensure that job credentials are available.

!echo | condor_store_cred add-oauth -s scitokens -i - >/dev/null


## Remove log files from previous runs.

pathlib.Path("convert_dataset.log").unlink(missing_ok=True)
pathlib.Path("error").mkdir(exist_ok=True)
pathlib.Path("output").mkdir(exist_ok=True)

for path in pathlib.Path("error").glob("*.err"):
    path.unlink()

for path in pathlib.Path("output").glob("*.out"):
    path.unlink()


## Submit the jobs.

itemdata = [
    {"aws_object": path, "aws_filename": os.path.basename(path)}
    for path
    in aws_objects
]
output_dir = "visus"

job_description = htcondor.Submit(
    f"""
    container_image = {container_image}
    args = python3 convert_dataset.py $(aws_filename) {output_dir}

    transfer_input_files = \
        convert_dataset.py, \
        osdf:///aws-opendata/us-west-2/nex-gddp-cmip6/$(aws_object), \
        osdf.token
    transfer_output_files = \
        {output_dir}/$(aws_filename), \
        {output_dir}/$(aws_filename).idx
    transfer_output_remaps = " \
        {output_dir}/$(aws_filename) = {output_loc}/nex-gddp-cmip6; \
        {output_dir}/$(aws_filename).idx = {output_loc}/$(aws_filename).idx"

    ## Specify resource requests and other requirements.
    request_cpus = 2
    request_memory = 4G
    request_disk = 4G

    ## Save the job log, and standard output and error.
    log = convert_dataset.log
    output = output/convert_dataset.$(JobId).out
    error = error/convert_dataset.$(JobId).err

    ## Make it easier to monitor and follow-up on "failed" jobs.                              
    on_exit_hold = ExitCode =!= 0

    ## Use the backfill EP provided by the OSPool Notebooks service.
    # +FromJupyter = true
    # requirements = Machine == "CHTC-Jupyter-User-EP.{platform.node()}"
    """
)

submitted_job = htcondor.Schedd().submit(job_description, itemdata=iter(itemdata))

### Wait for the job to complete

In [ ]:
import demo_support

demo_support.wait_for_job(f"convert_dataset.log")

### (WIP) Upload a single OpenVisus idx file for the entire dataset

The Pelican `fsspec` implementation currently does not support writing objects.

In [ ]:
!PELICAN_LOGGING_DISABLEPROGRESSBARS=true pelican object put --debug --token osdf.token ../nex-gddp-cmip6-listing/nex-gddp-cmip6.idx {dataset_loc}

In [ ]:
import pathlib

token = pathlib.Path("osdf.token").read_text()

osdf_fs.put(
    "../nex-gddp-cmip6-listing/nex-gddp-cmip6.idx",
    dataset_path,
    headers={"Authorization": f"Bearer {token}"}
)

## Visualize the dataset

### Import libraries

In [ ]:
import IPython.display
import matplotlib.animation
import matplotlib.pyplot as plt
import numpy as np
import OpenVisus
import openvisuspy as ov

### Load the dataset

In [ ]:
db = ov.LoadDataset(dataset_loc + "?directread")

Note: Pelican's `directread` option here should only be necessary when there are problems with its internal caching infrastructure.

### Show some basic information

In [ ]:
print("Dimensions:", db.getLogicBox())
print("Total Timesteps:", len(db.getTimesteps()))
print("Total Field:", len(db.getFields()))

### Animate the data

Only a few timestemps are chosen (see the definition of `animation`) for the purposes of illustration.

In [ ]:
## Extract dimensions from the first timestep.

quality = 0  # full resolution = 0, coarse = -4, coarser = -8
timestep = db.getTimesteps()[0]
data3D = db.db.read(time=timestep, quality=quality)
data = data3D[:,:]
H, W = data3D.shape


## Define and show the animation.

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10 * H / W))
axes.set_xlim(0, W)
axes.set_ylim(0, H)
# TODO: Determine how to set `vmin` and `vmax`.
image = axes.imshow(data, extent=[0, W, 0, H], aspect="auto", origin="lower", vmax=110, cmap="viridis")

def frame_fn(timestep):
    data3D = db.db.read(time=timestep, quality=quality)
    data = data3D[:,:]
    image.set_data(data)

plt.rcParams["animation.embed_limit"] = 100  # MB
animation = matplotlib.animation.FuncAnimation(fig, frame_fn, frames=db.getTimesteps()[4:8])
IPython.display.HTML(animation.to_jshtml())